<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clean-the-'city'-column" data-toc-modified-id="Clean-the-'city'-column-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clean the 'city' column</a></span></li><li><span><a href="#Clean-the-'zip'-column" data-toc-modified-id="Clean-the-'zip'-column-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Clean the 'zip' column</a></span></li><li><span><a href="#Clean-the-'SendFacture'-column" data-toc-modified-id="Clean-the-'SendFacture'-column-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Clean the 'SendFacture' column</a></span></li><li><span><a href="#Prepare-date-to-stack-overflow" data-toc-modified-id="Prepare-date-to-stack-overflow-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prepare date to stack overflow</a></span></li></ul></div>

In [ ]:
import pandas as pd
from pandas import Timestamp
import re
import matplotlib.pyplot as plt
import datetime


In [ ]:
path_input='./../data/data_raw/'
path_output='./../data/data_created/' 
#path =''
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
#lod file
df_fact_raw = pd.read_excel(path_input+'factures.xls')

In [ ]:
df_fact_raw.describe()

In [ ]:
df_fact_raw.info()


In [ ]:
df_fact_raw.head(2)

In [ ]:
df_fact = df_fact_raw.copy()
df_fact.drop('Adresse du client', axis=1, inplace=True)

In [ ]:

df_fact= df_fact.rename(columns={'Numero de facture': 'FactureID', \
                        'Titre': 'Subscription Period' , \
                        'Montant HT': 'HT', \
                        'Montant TTC' : 'TTC', \
                        'Taux de TVA' : 'TxTva', \
                        'Montant TVA': 'TVA', \
                        'Facture envoyée le': 'SendFacture' ,\
                        'Code postal du client' : 'Zip', \
                        'Ville du client' : 'City'})
df_fact['Zip'] = df_fact['Zip'].fillna(0).astype(int)

In [ ]:
df_fact = df_fact.sort_values(by='Client', ascending=False)

In [ ]:
df_fact.loc[55, 'Subscription Period'] = 'Abonnement Arsenal chirurgical : du 01/07/2019 au 30/06/2020'


In [ ]:
df_fact.loc[90, 'HT'] = 5400
df_fact.drop(93, axis=0, inplace=True)

#### Clean the 'city' column

In [ ]:
df_fact.loc[df_fact.Client.str.contains(r'AMIENS'), ["Zip", "City"]] = 80000, 'Amiens'
df_fact.loc[df_fact.Client.str.contains(r'ELSAN NANTES'), ["Zip", "City"]] = 44000, 'Nantes'

In [ ]:
df_fact['City'] = df_fact['City'].str.title().str.replace(r'(\bCedex\s?\d?\d?)',"" )

#### Clean the 'zip' column

In [ ]:
# fill with 0 at the start if note lenght 5
df_fact['Zip']=df_fact['Zip'].astype(str).str.zfill(5)

#### Clean the 'SendFacture' column

In [ ]:
# fill the NaT with the date 2000 01 01.
df_fact['SendFacture'] = df_fact['SendFacture'].fillna(Timestamp('20000101'))

In [ ]:
#df_fact = df_fact.sort_values(by='Client')
len(df_fact)

#### Prepare date to stack overflow

In [ ]:
def clean_Subscription(df_fact):
    # clean if date begin and end in SubscriptionPeriod
    # create 3 new colonnes
    df_period = df_fact['Subscription Period'] \
        .str.extractall(r'(?P<Period>(?P<Begin>(0[1-9]|[12][0-9]|3[01])[/](0[1-9]|1[012])[/](19|20)?\d\d).+(?P<End>(0[1-9]|[12][0-9]|3[01])[/](0[1-9]|1[012])[/](19|20)?\d\d))')
    df_fact['Period'] = df_period['Period'].unstack()
    df_fact['Begin'] = df_period['Begin'].unstack()
    df_fact['Begin'] = df_fact['Begin'].str.replace(r'((\d{2})/(\d{2})/(\d{2,4}))', r'\4-\3-\2' )
    df_fact['End'] = df_period['End'].unstack()
    df_fact['End'] = df_fact['End'].str.replace(r'((\d{2})/(\d{2})/(\d{2,4}))', r'\4-\3-\2' )
    
    return(df_fact)

    

In [ ]:
df_clean = clean_Subscription(df_fact)

In [ ]:
begin2 = df_clean['Subscription Period'].str.extract(r'(\d{2}/\d{2}/\d{2,4})[)]?$').rename({0:'Begin2'}, axis=1)
begin2 = begin2.Begin2.str.replace(r'((\d{2})/(\d{2})/(\d{2,4}))', r'\4-\3-\2' )
df_clean = pd.concat([df_clean, begin2], axis=1)
df_clean.Begin = df_clean.Begin.combine_first(df_clean.Begin2)
df_clean = df_clean.drop('Begin2', axis=1)

In [ ]:
begin3 = df_clean['Subscription Period'].str.extract(r'(\d{2}\s\w+\s\d{2,4})[)]?$').rename({0:'Begin3'}, axis=1)
df_clean = pd.concat([df_clean, begin3], axis=1)

In [ ]:
df_clean.Begin = df_clean.Begin.combine_first(df_clean.Begin3)

In [ ]:

df_clean = df_clean.drop('Begin3', axis=1)

In [ ]:
df_clean['Begin'].replace(r'01 janvier 2019', '2019-01-01', inplace=True)
df_clean['Begin'].replace(r'01 octobre 2018', '2018-10-01', inplace=True)

In [ ]:
df_clean['Date'] = df_clean['Date'].astype(str)

In [ ]:
df_clean.Begin = df_clean.Begin.combine_first(df_clean.Date)

In [ ]:
df_clean['Begin'].replace(r'01 janvier 2019', '2019-01-01', inplace=True)
df_clean['Begin'].replace(r'01 octobre 2018', '2018-10-01', inplace=True)
df_clean['Begin'].replace(r'18-11-22', '2018-11-22', inplace=True)
df_clean['End'].replace(r'18-12-31', '2018-12-31', inplace=True)

In [ ]:
df_clean['Begin'] = pd.to_datetime(df_clean['Begin'], format ='%Y-%m-%d')
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format ='%Y-%m-%d')
df_clean['End'] = pd.to_datetime(df_clean['End'], format ='%Y-%m-%d')

In [ ]:
End2 = df_clean['Begin'] + pd.offsets.DateOffset(years=1) -  pd.offsets.DateOffset(days=1)

In [ ]:
End2 = End2.to_frame()

In [ ]:
End2.rename({'Begin': 'End2'}, axis=1, inplace=True)

In [ ]:
df_clean = pd.concat([df_clean, End2], axis=1)

In [ ]:
df_clean.End = df_clean.End.combine_first(df_clean.End2)
df_clean = df_clean.drop('End2', axis=1)

In [ ]:
df_clean.info()

In [ ]:
df_clean = df_clean.drop(['Subscription Period', 'FactureID', 'SendFacture', 'Period'], axis=1)

In [ ]:
df_clean.info()

In [ ]:
df_clean['Week_Number_Begin'] = df_clean['Begin'].dt.week
df_clean['Year_Begin'] = df_clean['Begin'].dt.year
df_clean[df_clean[['Client', 'Year_Begin']].duplicated(keep=False)]

In [ ]:
df_clean.to_csv(path_output+'HistoryCa.csv')